# Expand array (UNNEST)

Sometimes you need to deal with arrays of arrays or, more scientifically, a column containing arrays in each record.

So you can easily perform this operation using the `UNNEST(<column>)` syntax after the `SELECT` keyword.

## Using example

So in the following cell I'll create an example database that will be used in the cells below to show the properties of the keyword `UNNEST`.

In [1]:
%%bash
docker run --rm -d\
    -e POSTGRES_PASSWORD=postgres \
    --name unnest_example_postgres \
    postgres:15.4 &> /dev/null
sleep 5
docker exec -i unnest_example_postgres psql -U postgres -d postgres

CREATE TABLE example_table(
    col1 TEXT,
    col2 INT[],
    col3 TEXT[]
);

INSERT INTO example_table(col1, col2, col3) VALUES
('A', '{1,2,3}', '{"hello", "fedor"}'),
('B', '{3,2}', '{"test"}'),
('C', '{3}', '{"value, with, commas", "value with spaces"}'),
('D', '{3,4,5}', '{"final_element"}');

SELECT * FROM example_table;

CREATE TABLE
INSERT 0 4
 col1 |  col2   |                    col3                     
------+---------+---------------------------------------------
 A    | {1,2,3} | {hello,fedor}
 B    | {3,2}   | {test}
 C    | {3}     | {"value, with, commas","value with spaces"}
 D    | {3,4,5} | {final_element}
(4 rows)



**Note** don't forget to stop the container when you've finished playing with it.

In [72]:
!docker stop unnest_example_postgres

unnest_example_postgres


## Basic example

So here I just using unnest for some for some arbitrary column. The values of other columns will be duplicated so that each element of the unnested array corresponds to an element that corresponded to the original record.

In [62]:
%%bash
docker exec -i unnest_example_postgres psql -U postgres -d postgres

SELECT col1, UNNEST(col2), col3 FROM example_table;

 col1 | unnest |                    col3                     
------+--------+---------------------------------------------
 A    |      1 | {hello,fedor}
 A    |      2 | {hello,fedor}
 A    |      3 | {hello,fedor}
 B    |      3 | {test}
 B    |      2 | {test}
 C    |      3 | {"value, with, commas","value with spaces"}
 D    |      3 | {final_element}
 D    |      4 | {final_element}
 D    |      5 | {final_element}
(9 rows)



## Multicolumn unnest

Using multiple UNNESTs in the same query leads to unexpected results for me. Not separate records for each combination of array elements, but records with pairwise matches, with skips in case arrays for the same record are of different lengths.

In [60]:
%%bash
docker exec -i unnest_example_postgres psql -U postgres -d postgres

SELECT col1, UNNEST(col2), UNNEST(col3) FROM example_table;

 col1 | unnest |       unnest        
------+--------+---------------------
 A    |      1 | hello
 A    |      2 | fedor
 A    |      3 | 
 B    |      3 | test
 B    |      2 | 
 C    |      3 | value, with, commas
 C    |        | value with spaces
 D    |      3 | final_element
 D    |      4 | 
 D    |      5 | 
(10 rows)



If you want to get a separate record for each combination of unnested values, just use subquery (at least I haven't found a better solution yet).

So in the following example, there is a separate record for each combination of values in the corresponding `col2` and `col3` records.

In [3]:
%%bash
docker exec -i unnest_example_postgres psql -U postgres -d postgres

SELECT col1, col2, UNNEST(col3)
FROM (SELECT 
        col1, UNNEST(col2) AS col2, col3 
    FROM example_table
) AS t;

 col1 | col2 |       unnest        
------+------+---------------------
 A    |    1 | hello
 A    |    1 | fedor
 A    |    2 | hello
 A    |    2 | fedor
 A    |    3 | hello
 A    |    3 | fedor
 B    |    3 | test
 B    |    2 | test
 C    |    3 | value, with, commas
 C    |    3 | value with spaces
 D    |    3 | final_element
 D    |    4 | final_element
 D    |    5 | final_element
(13 rows)

